In [1]:
import sys
import functools
from functools import reduce
import numpy as np
import pandas as pd
import re
import functools
import subprocess
import os
import pickle
import math as m
import fileinput
log_name = "log.log"
import mysql.connector 
from mysql.connector import Error
import shutil
mysql_dir = "/var/lib/mysql-files/"
from tqdm import tqdm
libz_ = './../../libs/'
sys.path.append('./../../libs/')
initdir = os.getcwd() +'/'
os.chdir('./../../libs/')
from make import make_dir,recover_make,all_,comp_,clang_flags,make_prg,make_routine,opts_lvl,clangs,opt_line,opt_init_line,ch_opt_string
from make import opt_routine,targ_routine,phony_routine,clean_routine,select_groups,mopt_routine,reg_name,mopt_routine,concate_dir,replace_suffixroutine
from sql_connection import mysql_connection
from sel_feature import diff_pass
os.chdir(initdir)    

optdir = "./../../opt_optimzations/"
opts = ['opt_O%s' % i for i in ['1','2','3','s','z']]
clangs = ['clang-O%s' % i for i in ['1','2','3','s','z']]



large_dir = initdir+"./../../../Unix_Source/unix_cmp/"
small_dir = initdir+"./../../../Unix_Source/unix_small_2/"

In [2]:
opt_file = {'opt_O1':{}, 'opt_O2':{}, 'opt_O3':{}, 'opt_Os':{}, 'opt_Oz':{}}
frame_cols = set()
ind_cols = {}
opt_lvl_str = ['start','main','end','final']
os.chdir(initdir+optdir)
for f in opts_lvl:
    if (os.path.isfile(f)) and (os.path.splitext(f)[1]==''):
        ind = 0
        with open(f,'rt') as op_fl:
            ind_cols[f] = {}
            for line in op_fl:
                opt_str = re.split(r'Pass Arguments\:\s+',line)[1][:-1]
#                 print(opt_str)
                opt_file[f][opt_lvl_str[ind]] = opt_str
                ind_cols[f] = reduce(lambda x,y:concate_dir(x,y)
                                                ,opt_str.split(' '),ind_cols[f])
                ind+=1
                frame_cols = frame_cols.union(opt_str.split(' '))
                

In [3]:
opt_frame = pd.DataFrame.from_dict(ind_cols).T.replace(np.NAN,0)
opt_frame

,-tti,-tbaa,-scoped-noalias,-assumption-cache-tracker,-targetlibinfo,-verify,-ee-instrument,-simplifycfg,-domtree,-sroa,...,-div-rem-pairs,-inline,-mldst-motion,-gvn,-elim-avail-extern,-slp-vectorizer,-constmerge,-callsite-splitting,-argpromotion,-aggressive-instcombine
opt_O1,2.0,2.0,2.0,2.0,4.0,2.0,1.0,8.0,13.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
opt_O2,2.0,2.0,2.0,2.0,4.0,2.0,1.0,8.0,13.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
opt_O3,2.0,2.0,2.0,2.0,4.0,2.0,1.0,8.0,13.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
opt_Os,2.0,2.0,2.0,2.0,4.0,2.0,1.0,8.0,13.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
opt_Oz,2.0,2.0,2.0,2.0,4.0,2.0,1.0,8.0,13.0,2.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [5]:
def examine_str(options=['-O1'],output_='pasta.bc',input_ = ll_file):
    cmd = ['opt','-debug-pass=Arguments'] + options + [input_]
    proc = subprocess.Popen(cmd,stderr=subprocess.PIPE,stdout=subprocess.PIPE)
    o,e = proc.communicate()
    if not proc.returncode:
        return e
    else:
        return e.decode('utf-8')

In [112]:
os.chdir(initdir)
passes = {}
for pass_ in opt_frame.columns:
    cmd = ['opt','-debug-pass=Arguments',pass_,ll_file,'-o',pass_+ll_file[:-3]+'.bc']
    proc = subprocess.Popen(cmd,stderr=subprocess.PIPE,stdout=subprocess.PIPE)
    o,e = proc.communicate()
    if not proc.returncode:
        passes[pass_] = re.split('Pass Arguments\:\s+',e.decode('ascii'))[1][:-1].split(' ')

# Opt level exemation shows independency between all target lvl(main), other three lvl in similar state (start,end,final)

In [47]:
for file in opts_lvl:
    for d_file in opts_lvl:
        for lvl in [opt_lvl_str[0]]+opt_lvl_str[2:]:
            if opt_file[file][lvl]!=opt_file[d_file][lvl]:
                print(lvl,file,d_file)

In [116]:
os.chdir(initdir)
cmd_lvl = {}
curr_opt_str = []
for opt in opt_file['opt_O1']['main'].split(' ')[1:]:
    curr_opt_str.append(opt)    
    cmd_lvl[opt] = examine_str(curr_opt_str+[ll_file,'-o','tmp.bc'])
    os.system('rm -rf tmp.bc')

In [4]:
os.chdir(initdir)
passes = pickle.load(open('passes.dmp','rb'))

os.chdir(initdir)
cmd_lvl = pickle.load(open('cmd.lvl','rb'))

keys = list(cmd_lvl.keys())
repeated_options = set(cmd_lvl[keys[0]].split(' '))
for opt_ in keys[1:]:
    repeated_options.intersection(cmd_lvl[opt_].split(' '))
repeated_options.remove(keys[0])
for opt_ in keys:
    cmd_lvl[opt_] = re.sub(r'\s+',' ',reduce(lambda x,y : x.replace(y,''),list(repeated_options),cmd_lvl[opt_]))[1:-1]
for pass_ in passes:
    for rp_opt in list(repeated_options):
        passes[pass_].remove(rp_opt)
for file in opt_file:
     opt_file[file]['main'] = (reduce(lambda x,y : x.replace(y,''),
                  list(repeated_options),opt_file[file]['main'])[1:-1].split(' '))

# Not include tough speed

In [5]:
grow_speed = {}
prev_speed = cmd_lvl[keys[0]].split(' ').__len__()
for ind,opt_ in enumerate(keys[1:]):
    if (len(cmd_lvl[opt_].split(' '))-prev_speed)!=1:
        grow_speed[opt_] = {'speed':cmd_lvl[opt_].split(' ').__len__(),'prev':keys[ind],'curr':ind+1}
    prev_speed = len(cmd_lvl[opt_].split(' '))

# Detecting pass groups

In [5]:
pass_presence = {}
for pass_ in passes:
    pass_presence.setdefault(pass_,[])
    targ_pass = set(passes[pass_])
    ###second cycle
    for itt_pass in passes:
        if itt_pass!=pass_:
            dest_pass = set(passes[itt_pass])
            if targ_pass == targ_pass.intersection(dest_pass):
                pass_presence[pass_].append(itt_pass)
    if pass_presence[pass_].__len__()==0:
        del pass_presence[pass_]
            


# Build order options 

In [5]:
os.chdir(initdir+libz_)
opt_order = pickle.load(open('order.dmp','rb'))
true_order = pickle.load(open('true_order.dmp','rb'))
intr_b_ents = pickle.load(open('trv.dmp','rb'))
os.chdir(initdir)



# opt_order = {} 
# for opt_ in opt_file:
#     build_pass = opt_file[opt_]['main']
#     is_build = set()
#     tmp = []
#     opt_order.setdefault(opt_,{})
#     ind = 0
#     for _len,proc_pass in enumerate(build_pass):
#         if proc_pass not in is_build:
#             is_build.add(proc_pass)
#             opt_order[opt_][ind] = {'pos':_len,
#                               'pass':proc_pass,
#                               'pass_str':' '.join(build_pass[:_len+1]),
#                               'in_complete':' '.join(tmp)}
#             tmp = []
#             ind+=1
#         else:
#             tmp.append(proc_pass)

# intr_b_ents = {}
# for lvl_ent in range(1,len(opts_lvl)):
#     intr_b_ents[opts_lvl[lvl_ent]] = {}
#     for oth_entity in opts_lvl[:lvl_ent]:
#         ls = []
#         for ind in range(len(opt_order[opts_lvl[lvl_ent]])):
#             if opt_order[opts_lvl[lvl_ent]][ind]['pass'] != opt_order[oth_entity][ind]['pass']:
#                 break
#             else:
#                 ls.append(opt_order[opts_lvl[lvl_ent]][ind]['pass'])
#         intr_b_ents[opts_lvl[lvl_ent]][oth_entity] = (ls,len(ls))
# intr_b_ents['opt_O1'] = {'opt_O1':([],0)}    

# true_order = {}
# for lvl in opt_order:
#     true_order[lvl] = {}
#     size = 0
#     for _lvl in intr_b_ents[lvl]:
#         if size < intr_b_ents[lvl][_lvl][1]:
#             size = intr_b_ents[lvl][_lvl][1]
#     for ind in  range(size,len(opt_order[lvl])):
#         true_order[lvl][ind] = {'pass':(opt_order[lvl][ind]['pass'])}


# LARGE FILES

In [9]:
related_ll = {}
os.chdir(large_dir)
for file in os.listdir():
    os.chdir(large_dir)
    if os.path.isdir(file):
        dir_ = make_dir("{}/{}/".format(file,'source'))[0]
        os.chdir(dir_)
        related_ll[file] = {}
        with open('Makefile','r') as make_file:
            ls = make_file.readlines()
            ind = 0
            while (ind!=len(ls)):
                if re.match('TARGET\_T.+\=.*',ls[ind]):
                    numb_ = re.split('\s*\=\s*',ls[ind][:-1])[0].split('_T')[1]
                    asm = re.split('\s*\=\s*',ls[ind][:-1])[1]
                    related_ll[file][numb_] = {'asm':asm,'lls':[]}
                elif re.match('OBJS\_T.+\=.*',ls[ind]):
                    obj_str = ""
                    numb_ = re.split('\s*\=\s*',ls[ind][:-1])[0].split('_T')[1]
                    while(ls[ind]!='\n'):
                        ind+=1 
                        obj_str+=ls[ind]
                    related_ll[file][numb_]['lls']+=(list(map(lambda x : os.path.splitext(x)[0]+'.ll',re.sub("\s+"," ",
                     obj_str.replace("\\",'').replace('\t','').replace('\n',''))[:-1].split(' '))))
                ind+=1

In [116]:
conn = mysql_connection()
cursor = conn.cursor()
for file in related_ll:
    cursor.execute("INSERT INTO PG (PG_NAME) VALUES ('%s')" %file)
    conn.commit()
    cursor.execute("SELECT ID FROM PG WHERE PG_NAME='%s'" %file)
    pg_id = cursor.fetchone()[0]
    target_id = cursor.fetchone()
    for target in related_ll[file]:
        trg = related_ll[file][target]['asm']
        cursor.execute("INSERT INTO TARGET (PG_ID,TARGET_NAME) VALUES (%s,'%s')" % (pg_id,trg))
        conn.commit()
        cursor.execute("SELECT ID FROM TARGET WHERE PG_ID=%s AND TARGET_NAME='%s'" % (pg_id,trg))
        trg_id = cursor.fetchone()[0]
        for ll in related_ll[file][target]['lls']:
            cursor.execute("INSERT INTO RL_LL (TARGET_ID,LL_NAME) VALUES (%s,'%s')" %(trg_id,ll) )
        conn.commit()
conn.close()
        
        

Connected to MySQL Server version  5.7.30-0ubuntu0.18.04.1


# SMALL_FILES

In [30]:
related_ll = {}
for file in os.listdir(small_dir):
    os.chdir(small_dir)
    if os.path.isdir(file):
        dir_ = make_dir("{}/{}/".format(file,'source'))[0]
        os.chdir(dir_)
        related_ll[file] = {}
        with open('Makefile','r') as make_file:
            for line in make_file:
                if (re.match(r'all\s*\:\s*',line)):
                    related_ll[file][1] = {'asm':re.split('all\s*\:\s*',line[:-1])[1]
                                           ,'lls':[re.split('all\s*\:\s*',line[:-1])[1]+'.ll']}

In [18]:
conn = mysql_connection()
cursor = conn.cursor()
for file in related_ll:
    cursor.execute("INSERT INTO PG (PG_NAME) VALUES ('%s')" %file)
    conn.commit()
    cursor.execute("SELECT ID FROM PG WHERE PG_NAME='%s'" %file)
    pg_id = cursor.fetchone()[0]
    target_id = cursor.fetchone()
    for target in related_ll[file]:
        trg = related_ll[file][target]['asm']
        cursor.execute("INSERT INTO TARGET (PG_ID,TARGET_NAME) VALUES (%s,'%s')" % (pg_id,trg))
        conn.commit()
        cursor.execute("SELECT ID FROM TARGET WHERE PG_ID=%s AND TARGET_NAME='%s'" % (pg_id,trg))
        trg_id = cursor.fetchone()[0]
        for ll in related_ll[file][target]['lls']:
            cursor.execute("INSERT INTO RL_LL (TARGET_ID,LL_NAME) VALUES (%s,'%s')" %(trg_id,ll) )
        conn.commit()
conn.close()
        
        

##  Mysql routine

In [125]:
    
conn = mysql_connection()
cursor = conn.cursor()
for option in opt_frame.columns:
    cursor.execute("INSERT INTO OPTIONS (NAME) VALUES ('%s')" %option)
conn.commit()
for opt_lvl in opt_order:
    for num in opt_order[opt_lvl]:
        pass_ = opt_order[opt_lvl][num]['pass']
        cursor.execute("SELECT ID FROM OPTIONS WHERE NAME='%s'" %pass_)
        pass_id = cursor.fetchone()[0]
        cursor.execute("INSERT INTO OPTS_LVL (NAME,OPT_ID) VALUES ('%s',%s)" %(opt_lvl,pass_id))
conn.commit()
conn.close()


Connected to MySQL Server version  5.7.30-0ubuntu0.18.04.1


In [10]:
file = {}
conn = mysql_connection(db_='files')
cursor = conn.cursor()
init_file = os.listdir(small_dir)[0]
if os.path.isdir(small_dir+init_file):
    cursor.execute("SELECT ID FROM TARGET WHERE PG_ID = (SELECT ID FROM PG WHERE PG_NAME='%s')" %init_file)
    targ_id = cursor.fetchone()[0]
    cursor.execute("SELECT LL_ID FROM RL_LL WHERE TARGET_ID=%s" %(targ_id))
    ll_id = cursor.fetchone()[0]
    cursor.execute("SELECT DATA FROM ETALON_DATA WHERE RL_LL_ID=%s" %(ll_id))
    file[-1] = {'file':cursor.fetchone()[0],'size':0}
    for lvl_opt in opt_order:
        for num in true_order[lvl_opt]['order']:
            cursor.execute("SELECT ID FROM OPTS_LVL WHERE NAME='%s' AND OPT_ID=(SELECT ID FROM OPTIONS WHERE NAME='%s')" %(lvl_opt,num))
            id_pass = cursor.fetchone()[0]
            cursor.execute("SELECT SIZE FROM TARGET_RES WHERE TARGET_ID=%s AND OPT_LVL_ID=%s" %(targ_id,id_pass))
            file[id_pass] ={'size':cursor.fetchone()[0]}
            cursor.execute("SELECT APPLIED_DATA FROM OPTS_DATA WHERE DATA_ID = (SELECT DATA_ID FROM MAIN_QUERY WHERE RL_LL_ID=%s and OPT_LVL_ID=%s)" %(ll_id,id_pass)) 
            file[id_pass] = {'file':cursor.fetchone()[0]}
    
# os.chdir(initdir)
# pickle.dump(file,open('structure_ll.dmp','wb'))

In [11]:
size_d = {}
for opt_lvl in intr_b_ents:
        size_d[opt_lvl] = {}
        try:
            key = (max(intr_b_ents[opt_lvl], key = lambda k: intr_b_ents[opt_lvl][k][0].__len__()))
            size_d[opt_lvl] = {'key':key,'size':intr_b_ents[opt_lvl][key][0].__len__()}
        except ValueError :
            size_d[opt_lvl] = {'size':0}
lvl_b = {}
keys = list(file.keys())
ind = -1
for i in range(1,keys.__len__()):
    if keys[i-1]+1<keys[i]:
        ind+=1
        lvl_b[opts[ind]] = keys[i]
        
prv_ind = {}
for lvl_opt in size_d:
    if 'key' in size_d[lvl_opt].keys():
        prv_ind[lvl_opt] = lvl_b[size_d[lvl_opt]['key']]+size_d[lvl_opt]['size']-1-size_d[size_d[lvl_opt]['key']]['size']
prv_ind['opt_O1'] = -1

fres = {}
outpass_ = {}
p_c = 2
for lvl_opt in opts_lvl:
    fres[lvl_opt] = {}
    outpass_[lvl_opt] = {}
    file[lvl_b[lvl_opt]-1] = file[prv_ind[lvl_opt]]
    p_c += size_d[lvl_opt]['size'] 
    print(p_c)
    for _opt in (true_order[lvl_opt]['order']):
        fres[lvl_opt][_opt],outpass_[lvl_opt][_opt] = diff_pass(file[p_c-1]['file'].splitlines(),file[p_c]['file'].splitlines(),str(p_c-1),str(p_c))
        p_c+= 1
            

2
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
97
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
157
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncountable line
Uncou

## DELETE ALL SMALLFILES

In [25]:
# conn = mysql_connection(db_='files')
# cursor = conn.cursor()
# for file in os.listdir(small_dir):
#     if os.path.isdir(small_dir+file):
#         cursor.execute("SELECT ID FROM TARGET WHERE PG_ID = (SELECT ID FROM PG WHERE PG_NAME='%s')" %file)
#         targ_id = cursor.fetchone()[0]
#         cursor.execute("DELETE FROM MAIN_QUERY WHERE RL_LL_ID = (SELECT LL_ID FROM RL_LL WHERE TARGET_ID=%s)" %(targ_id))
#         conn.commit()
#         cursor.execute("DELETE FROM TARGET_RES WHERE TARGET_ID=%s" %targ_id)
#         conn.commit()
# conn.close()

## DELETE ALL LARGEFILES

In [46]:
# conn = mysql_connection(db_='files')
# cursor = conn.cursor()
# for file in os.listdir(large_dir):
#     if os.path.isdir(large_dir+file):
#         cursor.execute("SELECT ID FROM TARGET WHERE PG_ID = (SELECT ID FROM PG WHERE PG_NAME='%s')" %file)
#         for targ_id, in cursor.fetchall():
#             cursor.execute("DELETE FROM MAIN_QUERY WHERE RL_LL_ID IN (SELECT LL_ID FROM RL_LL WHERE TARGET_ID=%s)" %(targ_id))
#             conn.commit()
#             cursor.execute("DELETE FROM TARGET_RES WHERE TARGET_ID=%s" %targ_id)
#             conn.commit()
# conn.close()

In [17]:
# replace_suffixroutine(large_dir,["\topt $(OPT_INITFLAG) $< -f | llvm-dis -o $*_new.ll\n",
#                                 "\topt $(OPT_FLAG) $*_new.ll -o $*.bc\n",
#                                 "\tllvm-dis $*.bc -o $*_new.ll\n"])
# todo = [x for x in os.listdir(large_dir) if os.path.isdir(large_dir+x) ]

In [ ]:
conn = mysql_connection(db_='files')
cursor = conn.cursor(buffered=True)
for file in tqdm(todo):
    flag = False
    os.chdir(large_dir)
    cursor.execute("SELECT ID FROM PG WHERE PG_NAME='%s'" %file)
    pg_id = cursor.fetchone()[0]
    dir_ = make_dir("{}/{}/".format(file,'source'))[0]
    for lvl_opt in opts_lvl: 
        pass_opt = true_order[lvl_opt]['used']
        for _opt in (true_order[lvl_opt]['order']):
            pass_opt.append(_opt)
            ch_opt_string(pass_opt,large_dir+dir_,true_order[lvl_opt]['init'].split(' '))
            make_prg(['make'])
            for target in related_ll[file]:
                tg_name = related_ll[file][target]['asm']
                ###move ll files in /var/lib/mysql-files/
                res =subprocess.check_output("wc -c {}".format(tg_name), stderr=subprocess.STDOUT,shell=True)
                res = res.decode('utf-8').split(' ')[0]
                os.system("cp *.ll /var/lib/mysql-files/")
                for ll in related_ll[file][target]['lls']:
                    cursor.execute("SELECT LL_ID FROM \
                                  (SELECT  RL_LL.LL_ID `LL_ID`,TARGET.PG_ID `PG_ID`,\
                                   TARGET.TARGET_NAME `TARGET_NAME`,RL_LL.LL_NAME `LL_NAME`FROM RL_LL INNER JOIN TARGET ON TARGET.ID = RL_LL.TARGET_ID) `GATHER` \
                                   WHERE PG_ID=%s AND TARGET_NAME='%s' AND LL_NAME='%s'" %(pg_id,tg_name,ll))
                    ll_id = cursor.fetchone()[0]
                    if not flag:
                        cursor.execute("INSERT INTO ETALON_DATA (RL_LL_ID,DATA) VALUES(%s,LOAD_FILE('%s'))" %(ll_id,mysql_dir+ll))                    
                        conn.commit()
                    cursor.execute("SELECT ID FROM (SELECT lvl.ID,lvl.NAME `LVL_NAME`,op.NAME `OPT_NAME` FROM `OPTS_LVL` lvl INNER JOIN `OPTIONS` op ON op.ID = lvl.OPT_ID) `offmi` WHERE LVL_NAME='%s' AND OPT_NAME='%s'" %(lvl_opt,_opt))
            
                    opt_lvl_id = cursor.fetchone()[0]
                    cursor.execute("INSERT INTO MAIN_QUERY (RL_LL_ID,OPT_LVL_ID) VALUES (%s,%s)" %(ll_id,opt_lvl_id))
                    conn.commit()
                    cursor.execute("SELECT DATA_ID FROM MAIN_QUERY \
                                   WHERE RL_LL_ID=%s AND OPT_LVL_ID=%s" %(ll_id,opt_lvl_id))
                    data_id = cursor.fetchone()[0]
                    new_ll = mysql_dir+ll[:-3]+"_new.ll"
                    cursor.execute("INSERT INTO OPTS_DATA (DATA_ID,APPLIED_DATA) VALUES(%s,LOAD_FILE('%s'))" %(data_id,new_ll))
mv               conn.commit()
                cursor.execute("INSERT INTO TARGET_RES (TARGET_ID,OPT_LVL_ID,SIZE) VALUES ((SELECT ID FROM TARGET WHERE PG_ID=%s and TARGET_NAME='%s'),%s,'%s')" %(pg_id,tg_name,opt_lvl_id,res))
                conn.commit()
            make_prg(['make','clean'])    
            os.chdir(mysql_dir)
            os.system('rm -f *.ll')
            flag = True
conn.close()







  0%|          | 0/41 [00:00<?, ?it/s]





  2%|▏         | 1/41 [3:33:40<142:27:09, 12820.73s/it]





  5%|▍         | 2/41 [4:25:50<107:23:42, 9913.41s/it] 





  7%|▋         | 3/41 [4:53:09<78:26:27, 7431.24s/it] 





 10%|▉         | 4/41 [5:12:42<57:04:49, 5553.77s/it]





 12%|█▏        | 5/41 [8:32:09<74:46:32, 7477.57s/it]





 15%|█▍        | 6/41 [9:23:01<59:47:30, 6150.01s/it]





 17%|█▋        | 7/41 [10:27:35<51:38:01, 5467.09s/it]





 20%|█▉        | 8/41 [12:44:48<57:43:13, 6296.77s/it]





 22%|██▏       | 9/41 [13:01:15<41:48:50, 4704.07s/it]





 24%|██▍       | 10/41 [14:09:14<38:53:32, 4516.55s/it]





 27%|██▋       | 11/41 [18:57:01<69:30:43, 8341.46s/it]





 29%|██▉       | 12/41 [19:46:48<54:15:21, 6735.23s/it]





 32%|███▏      | 13/41 [27:41:22<103:06:33, 13256.90s/it]





 34%|███▍      | 14/41 [30:04:13<88:52:58, 11851.07s/it] 





 37%|███▋      | 15/41 [35:34:26<102:50:29, 14239.60s/it]





 39%|███▉      | 16/41 [40:11:05<103:48